In [3]:
#Imports 

import matplotlib as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [11]:
#datas
df=pd.read_csv('cleaned_datas.csv')
df

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv,Year
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,1997-02-28,1997,...,2.0,0,0,1,0,N,Y,P I F,60000.0,1997
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,1997-02-28,1997,...,2.0,0,0,1,0,N,Y,P I F,40000.0,1997
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,1997-02-28,1997,...,1.0,0,0,1,0,N,N,P I F,287000.0,1997
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,1997-02-28,1997,...,1.0,0,0,1,0,N,Y,P I F,35000.0,1997
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,1997-02-28,1997,...,1.0,7,7,1,0,N,N,P I F,229000.0,1997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,9995573004,FABRIC FARMS,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,1997-02-27,1997,...,1.0,0,0,1,0,0,N,P I F,70000.0,1997
899160,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,1997-02-27,1997,...,1.0,0,0,1,0,Y,N,P I F,85000.0,1997
899161,9995613003,"RADCO MANUFACTURING CO.,INC.",SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,1997-02-27,1997,...,1.0,0,0,1,0,N,N,P I F,300000.0,1997
899162,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,1997-02-27,1997,...,1.0,0,0,1,0,N,Y,CHGOFF,75000.0,1997


### First test based on interactive doc

In [ ]:
#Encoding categorical values with LabelEncoder
label_encoders = {}
categorical_cols = ['Zip', 'Name', 'City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc', 'MIS_Status']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  #Conversion in str type to avoid mistakes
    label_encoders[col] = le  # Stock the encoder if needed after for the prediction

#Selection of valuable features
features = ['Zip', 'NAICS', 'ApprovalFY', 'NewExist', 'CreateJob', 'RetainedJob', 
            'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc']

# Deletion of NaN values
df = df[features + ['MIS_Status']].dropna()


X = df[features] 
y = df['MIS_Status'] 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Logistic regression 
model = LogisticRegression(max_iter=100, solver='lbfgs')
model.fit(X_train, y_train)

#Predictions and values
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

#Result
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8225
Confusion Matrix:
[[     0  31509      0]
 [     1 147897      0]
 [     0    399      0]]
